![Title](./Novoenzyme_Enzyme_Stability_1.jpg)
<br>
<br>
![Title](./Novoenzyme_Enzyme_Stability_2.jpg)
<br>
<br>
![Title](./Novoenzyme_Enzyme_Stability_3.jpg)
<br>
<br>
![Title](./Novoenzyme_Enzyme_Stability_4.jpg)
<br>
<br>
![Title](./Novoenzyme_Enzyme_Stability_5.jpg)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from difflib import SequenceMatcher
from sklearn.model_selection import train_test_split
from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [2]:
novo = pd.read_csv('train.csv')
novo_update = pd.read_csv('train_updates_20220929.csv')

novo_update_seq_id = novo_update.pop('seq_id')
novo_update.dropna(how='all', inplace=True)
novo_update_ready = novo_update.merge(novo_update_seq_id, how='left', left_index=True, right_index=True)

novo.update(novo_update_ready)
novo['seq_id'] = novo['seq_id'].astype(int)
novo.set_index('seq_id', inplace=True)

print(novo.shape)
print(novo.columns)

(31390, 4)
Index(['protein_sequence', 'pH', 'data_source', 'tm'], dtype='object')


![Title](./Novoenzyme_Enzyme_Stability_6.jpg)

In [3]:
novo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31390 entries, 0 to 31389
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   protein_sequence  31390 non-null  object 
 1   pH                31104 non-null  float64
 2   data_source       28043 non-null  object 
 3   tm                31390 non-null  float64
dtypes: float64(2), object(2)
memory usage: 1.2+ MB


In [4]:
novo['pH'].describe()

count    31104.000000
mean         6.890323
std          1.588809
min          1.990000
25%          7.000000
50%          7.000000
75%          7.000000
max         64.900000
Name: pH, dtype: float64

![Title](./Novoenzyme_Enzyme_Stability_7.jpg)

In [5]:
novo = novo.dropna(subset = 'pH')

![Title](./Novoenzyme_Enzyme_Stability_8.jpg)

In [6]:
novo['pH'][novo['pH'] > 14].unique()

array([48.4, 49. , 55.6, 64.9, 39. , 53.4])

In [7]:
novo['pH'][novo['pH'] > 14] = novo['pH'][novo['pH'] > 14] * 0.1
novo['pH'] = novo['pH'].round(1)

C:\Users\marcj\AppData\Local\Temp\ipykernel_11640\2319005900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  novo['pH'][novo['pH'] > 14] = novo['pH'][novo['pH'] > 14] * 0.1


![Title](./Novoenzyme_Enzyme_Stability_9.jpg)

In [8]:
novo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31104 entries, 0 to 31389
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   protein_sequence  31104 non-null  object 
 1   pH                31104 non-null  float64
 2   data_source       27769 non-null  object 
 3   tm                31104 non-null  float64
dtypes: float64(2), object(2)
memory usage: 2.2+ MB


In [9]:
novo.corr()

C:\Users\marcj\AppData\Local\Temp\ipykernel_11640\609685233.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  novo.corr()


,pH,tm
pH,1.000000,0.072835
tm,0.072835,1.000000


In [10]:

novo['comp_score'] = np.nan

for value in range(0, len(novo)):
    comparison_score = SequenceMatcher(a = novo['protein_sequence'].iloc[value], b = novo['protein_sequence'].iloc[14318]).ratio()
    novo['comp_score'].iloc[value] = comparison_score
novo.head()

,protein_sequence,pH,data_source,tm,comp_score
seq_id,,,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,0.148297
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,0.153153
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,0.128244
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,0.080378
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,0.045991


In [11]:
novo['comp_score'].describe()

count    31104.000000
mean         0.142037
std          0.055970
min          0.005224
25%          0.099066
50%          0.139674
75%          0.180791
max          1.000000
Name: comp_score, dtype: float64

In [12]:
novo.corr()

C:\Users\marcj\AppData\Local\Temp\ipykernel_11640\609685233.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  novo.corr()


,pH,tm,comp_score
pH,1.000000,0.072835,-0.197563
tm,0.072835,1.000000,0.010359
comp_score,-0.197563,0.010359,1.000000


In [13]:
novo['mol_wt'] = np.nan

for value in range(0, len(novo)):
    proto_an = ProteinAnalysis(novo['protein_sequence'].iloc[value])
    novo['mol_wt'].iloc[value] = proto_an.molecular_weight()

C:\Users\marcj\AppData\Local\Temp\ipykernel_11640\3905470451.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  novo['mol_wt'].iloc[value] = proto_an.molecular_weight()


In [14]:
novo.corr()

C:\Users\marcj\AppData\Local\Temp\ipykernel_11640\609685233.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  novo.corr()


,pH,tm,comp_score,mol_wt
pH,1.000000,0.072835,-0.197563,0.064199
tm,0.072835,1.000000,0.010359,-0.060967
comp_score,-0.197563,0.010359,1.000000,-0.401423
mol_wt,0.064199,-0.060967,-0.401423,1.000000
